In [1]:
%pip install --upgrade scikit-learn
%pip install python-dotenv
%pip install influxdb-client
%pip install pandas
%pip install pyyaml
%pip install numpy
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import io
import os
import time
import yaml
import warnings
import pandas as pd
import json
import hashlib
import csv
from influxdb_client import InfluxDBClient
from influxdb_client.client.warnings import MissingPivotFunction
from dotenv import load_dotenv

import numpy as np
from functools import partial

# Suppress specific warnings
warnings.simplefilter("ignore", MissingPivotFunction)

# Load environment variables
load_dotenv()

False

In [3]:
df = pd.read_csv('dataset.csv')

In [4]:
df.head()

,Unnamed: 0,bssid,channel,host,iteration,reference_point,rssi,ssid,time,topic,xr,yr
0,NaN,70:A7:41:DC:8E:55,1,LAUREN,1,RP29,-92,DTE Staff,2024-10-17T10:19:36.0867022Z,esp/topic,989,1037
1,NaN,70:A7:41:DC:8E:55,1,LAUREN,10,RP29,-94,DTE Staff,2024-10-17T10:23:26.6782315Z,esp/topic,989,1037
2,NaN,70:A7:41:DC:8E:55,1,LAUREN,11,RP29,-94,DTE Staff,2024-10-17T10:23:51.8041404Z,esp/topic,989,1037
3,NaN,70:A7:41:DC:8E:55,1,LAUREN,12,RP29,-90,DTE Staff,2024-10-17T10:24:13.6954786Z,esp/topic,989,1037
4,NaN,70:A7:41:DC:8E:55,1,LAUREN,13,RP29,-91,DTE Staff,2024-10-17T10:24:38.6661326Z,esp/topic,989,1037


In [5]:
df.columns

Index(['Unnamed: 0', 'bssid', 'channel', 'host', 'iteration',
       'reference_point', 'rssi', 'ssid', 'time', 'topic', 'xr', 'yr'],
      dtype='object')

In [6]:
df = df.drop(columns=['Unnamed: 0','host', 'topic'])
# df['time'] = pd.to_datetime(df['time'])

In [7]:
df.sample(10)

,bssid,channel,iteration,reference_point,rssi,ssid,time,xr,yr
1456,72:A7:41:9C:77:CD,11,1,RP35,-49,DTE Student,2024-10-23T08:45:55.4447115Z,1678,1037
1265,70:A7:41:DC:8E:55,1,17,RP33,-91,DTE Staff,2024-10-23T09:03:00.9753853Z,1449,1037
824,70:A7:41:DC:77:CD,11,18,RP14,-66,DTE Staff,2024-10-23T03:22:23.4403879Z,759,1948
584,70:A7:41:DC:77:CD,11,17,RP8,-68,DTE Staff,2024-10-17T09:59:02.1430941Z,529,1749
1363,70:A7:41:DC:77:CD,11,16,RP34,-46,DTE Staff,2024-10-23T09:12:10.7500781Z,1449,1202
62,72:A7:41:9C:8E:55,1,13,RP30,-91,DTE IoT,2024-10-17T10:34:12.6214057Z,989,1202
1589,70:A7:41:DC:8E:55,1,7,RP38,-80,DTE Staff,2024-10-23T06:47:41.2670918Z,1908,1202
1893,70:A7:41:DC:8E:55,1,17,RP42,-70,DTE Staff,2024-10-23T07:16:55.9024935Z,2454,1202
327,72:A7:41:9C:77:CD,11,18,RP3,-64,DTE Student,2024-10-17T09:11:24.3090027Z,300,1749
790,70:A7:41:DC:77:CD,11,5,RP13,-74,DTE Staff,2024-10-23T03:08:58.387339Z,759,1749


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1984 entries, 0 to 1983
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   bssid            1984 non-null   object
 1   channel          1984 non-null   int64 
 2   iteration        1984 non-null   int64 
 3   reference_point  1984 non-null   object
 4   rssi             1984 non-null   int64 
 5   ssid             1984 non-null   object
 6   time             1984 non-null   object
 7   xr               1984 non-null   int64 
 8   yr               1984 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 139.6+ KB


In [9]:
df['bssid'].unique()

array(['70:A7:41:DC:8E:55', '72:A7:41:9C:8E:55', '70:A7:41:DC:77:CD',
       '72:A7:41:9C:77:CD'], dtype=object)

In [10]:
df['ssid'] = df['ssid'].replace('DTE IoT', 'DTE Staff')

In [11]:
df['ssid'].unique()

array(['DTE Staff', 'DTE Student'], dtype=object)

In [12]:
df['reference_point'].unique()

array(['RP29', 'RP30', 'RP37', 'RP4', 'RP5', 'RP6', 'RP1', 'RP10', 'RP2',
       'RP3', 'RP7', 'RP8', 'RP9', 'RP11', 'RP12', 'RP13', 'RP14', 'RP15',
       'RP16', 'RP17', 'RP18', 'RP19', 'RP20', 'RP31', 'RP32', 'RP33',
       'RP34', 'RP35', 'RP36', 'RP38', 'RP39', 'RP40', 'RP41', 'RP42',
       'RP49'], dtype=object)

In [13]:
import pandas as pd

df['bssid'] = df['bssid'].str.lower()

# BSSID mapping for classification
bssid_classification = {
    '70:a7:41:dc:8e:55': 'ap_dosen',     # Dosen (Staff)
    '72:a7:41:9c:8e:55': 'ap_dosen',     # Dosen (Student)
    '70:a7:41:dc:77:cd': 'ap_digilab',   # Digilab (Staff)
    '72:a7:41:9c:77:cd': 'ap_digilab'    # Digilab (Student)
}

# Map BSSID to AP classification
df['AP_class'] = df['bssid'].map(bssid_classification)

# Check if there are any NaN values (due to missing BSSID mapping)
missing_bssid = df[df['AP_class'].isna()]
if not missing_bssid.empty:
    print("These BSSIDs were not classified:")
    print(missing_bssid)

# Pivot the dataframe
pivot_df = df.pivot_table(index=['time', 'reference_point', 'iteration', 'ssid', 'bssid', 'channel', 'xr', 'yr'], columns='AP_class', values='rssi').reset_index()

# Rename columns for clarity
pivot_df.columns = ['time', 'reference_point', 'iteration', 'ssid', 'bssid', 'channel', 'xr', 'yr'] + ['rssi_' + col for col in pivot_df.columns[8:]]

# Show the result
print(pivot_df)

                              time reference_point  iteration         ssid  \
0     2024-10-17T08:38:31.9525624Z             RP1          3  DTE Student   
1     2024-10-17T08:39:38.6086206Z             RP1          1    DTE Staff   
2     2024-10-17T08:39:45.8326223Z             RP1          1  DTE Student   
3     2024-10-17T08:40:03.1574484Z             RP1          2  DTE Student   
4     2024-10-17T08:40:08.1701809Z             RP1          2    DTE Staff   
...                            ...             ...        ...          ...   
1979  2024-10-23T09:33:08.6450442Z            RP31         19    DTE Staff   
1980  2024-10-23T09:33:24.6120935Z            RP31         20  DTE Student   
1981  2024-10-23T09:33:27.6923272Z            RP31         20    DTE Staff   
1982  2024-10-23T09:33:30.6587622Z            RP31         20    DTE Staff   
1983  2024-10-23T09:33:33.6198492Z            RP31         20    DTE Staff   

                  bssid  channel    xr    yr  rssi_ap_digilab  

In [14]:
pivot_df.sample(10)

,time,reference_point,iteration,ssid,bssid,channel,xr,yr,rssi_ap_digilab,rssi_ap_dosen
5,2024-10-17T08:40:27.6315854Z,RP1,3,DTE Student,72:a7:41:9c:77:cd,11,300,1368,-62.0,NaN
545,2024-10-17T10:32:51.3198226Z,RP30,10,DTE Staff,70:a7:41:dc:77:cd,11,989,1202,-42.0,NaN
1284,2024-10-23T07:15:33.4996029Z,RP42,14,DTE Student,72:a7:41:9c:77:cd,11,2454,1202,-61.0,NaN
624,2024-10-17T10:42:50.5685733Z,RP37,10,DTE Student,72:a7:41:9c:77:cd,11,1908,1037,-52.0,NaN
632,2024-10-17T10:43:44.5315951Z,RP37,12,DTE Student,72:a7:41:9c:77:cd,11,1908,1037,-53.0,NaN
900,2024-10-23T03:36:01.6214617Z,RP16,4,DTE Student,72:a7:41:9c:77:cd,11,989,1368,-56.0,NaN
1608,2024-10-23T08:48:11.5122601Z,RP35,6,DTE Staff,72:a7:41:9c:8e:55,1,1678,1037,NaN,-82.0
514,2024-10-17T10:29:37.3696287Z,RP30,2,DTE Student,72:a7:41:9c:77:cd,11,989,1202,-42.0,NaN
1932,2024-10-23T09:28:15.0604651Z,RP31,8,DTE Staff,70:a7:41:dc:77:cd,11,1219,1037,-44.0,NaN
1212,2024-10-23T07:02:28.8180086Z,RP36,15,DTE Staff,72:a7:41:9c:8e:55,1,1678,1202,NaN,-86.0


In [15]:
# Save the final dataframe
pivot_df.to_csv('ui_data.csv', index=False)
print(f"Data saved to {'ui_data.csv'}")

# Print some statistics
print(f"Date range: {pivot_df['time'].min()} to {pivot_df['time'].max()}")
print(f"Total records: {len(pivot_df)}")
print(f"X-coordinate range: {pivot_df['xr'].min()} to {pivot_df['xr'].max()}")
print(f"Y-coordinate range: {pivot_df['yr'].min()} to {pivot_df['yr'].max()}")

Data saved to ui_data.csv
Date range: 2024-10-17T08:38:31.9525624Z to 2024-10-23T09:33:33.6198492Z
Total records: 1984
X-coordinate range: 300 to 2454
Y-coordinate range: 1037 to 2130
